## Importações e Configurações Globais

In [1]:
import random

# Configurações do ambiente e dos agentes
LARGURA = 5
ALTURA = 5
SUJEIRA_INICIAL = [(1, 2), (2, 3), (4, 4)]
OBSTACULOS = [(2, 2), (3, 3)]
POSICAO_INICIAL_AGENTE = (0, 0)
NUM_ITERACOES = 50
MEDIDA_DESEMPENHO = 'medida2'  # Escolha entre 'medida1' ou 'medida2'


## Definição das Classes Sala e Ambiente
### A classe Sala representa cada sala individual, com atributos para posição, estado (suja ou limpa) e se é um obstáculo.
### A classe Ambiente mantém um dicionário de salas, permitindo acesso rápido às salas por coordenadas.

In [2]:
# Classe que representa cada sala no ambiente
class Sala:
    def __init__(self, x, y, obstaculo=False):
        self.x = x
        self.y = y
        self.suja = False
        self.obstaculo = obstaculo

    def __repr__(self):
        estado = 'O' if self.obstaculo else ('S' if self.suja else 'L')
        return f"Sala({self.x}, {self.y}): {estado}"

# Classe que representa o ambiente do aspirador de pó
class Ambiente:
    def __init__(self, largura, altura, sujeira_inicial, obstaculos):
        self.largura = largura
        self.altura = altura
        self.salas = {}
        for x in range(largura):
            for y in range(altura):
                obstaculo = (x, y) in obstaculos
                sala = Sala(x, y, obstaculo)
                if (x, y) in sujeira_inicial:
                    sala.suja = True
                self.salas[(x, y)] = sala

    def estado_da_sala(self, x, y):
        sala = self.salas.get((x, y))
        if sala:
            return sala.suja
        else:
            return None  # Fora dos limites

    def limpar_sala(self, x, y):
        sala = self.salas.get((x, y))
        if sala:
            sala.suja = False

    def sala_obstaculo(self, x, y):
        sala = self.salas.get((x, y))
        if sala:
            return sala.obstaculo
        else:
            return True  # Tratar fora dos limites como obstáculo

    def todas_salas_limpas(self):
        return all(not sala.suja for sala in self.salas.values() if not sala.obstaculo)


## Definição das Classes dos Agentes
### A classe base Agente define métodos comuns e atributos como posição e desempenho.
### AgenteReativoSimples implementa o comportamento reativo básico.
### AgenteBaseadoEmModelo mantém um modelo interno (modelo_interno) e uma lista de salas para visitar (salas_para_visitar).

In [3]:
# Classe base para os agentes
class Agente:
    def __init__(self, ambiente, posicao_inicial):
        self.ambiente = ambiente
        self.x, self.y = posicao_inicial
        self.desempenho = 0

    def perceber(self):
        # Retorna o estado da sala atual (suja ou limpa)
        return self.ambiente.estado_da_sala(self.x, self.y)

    def agir(self):
        pass  # Implementado nas subclasses

# Agente Reativo Simples
class AgenteReativoSimples(Agente):
    def agir(self):
        estado = self.perceber()
        if estado:  # Se a sala estiver suja
            self.ambiente.limpar_sala(self.x, self.y)
            print(f"Agente limpou a sala ({self.x}, {self.y}).")
            self.desempenho += 1  # Medida 1: +1 por limpeza
        else:
            # Movimenta-se para uma sala adjacente aleatória
            movimentos = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            random.shuffle(movimentos)
            for dx, dy in movimentos:
                novo_x = self.x + dx
                novo_y = self.y + dy
                if not self.ambiente.sala_obstaculo(novo_x, novo_y):
                    self.x = novo_x
                    self.y = novo_y
                    print(f"Agente moveu-se para ({self.x}, {self.y}).")
                    break  # Moveu-se com sucesso
            self.desempenho -= 1  # Medida 2: -1 por movimento

# Agente Baseado em Modelos
class AgenteBaseadoEmModelo(Agente):
    def __init__(self, ambiente, posicao_inicial):
        super().__init__(ambiente, posicao_inicial)
        self.modelo_interno = {}
        self.salas_para_visitar = []

    def atualizar_modelo(self):
        # Atualiza o modelo interno com a sala atual
        self.modelo_interno[(self.x, self.y)] = self.perceber()
        # Adiciona salas adjacentes não visitadas à lista
        movimentos = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in movimentos:
            novo_x = self.x + dx
            novo_y = self.y + dy
            if (novo_x, novo_y) not in self.modelo_interno and not self.ambiente.sala_obstaculo(novo_x, novo_y):
                self.salas_para_visitar.append((novo_x, novo_y))

    def agir(self):
        estado = self.perceber()
        self.atualizar_modelo()
        if estado:  # Se a sala estiver suja
            self.ambiente.limpar_sala(self.x, self.y)
            print(f"Agente (modelo) limpou a sala ({self.x}, {self.y}).")
            self.desempenho += 1  # Medida 1: +1 por limpeza
        elif self.salas_para_visitar:
            # Move-se para a próxima sala na lista
            proxima_sala = self.salas_para_visitar.pop(0)
            self.x, self.y = proxima_sala
            print(f"Agente (modelo) moveu-se para ({self.x}, {self.y}).")
            self.desempenho -= 1  # Medida 2: -1 por movimento
        else:
            # Se não há mais salas para visitar, move-se aleatoriamente
            movimentos = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            random.shuffle(movimentos)
            for dx, dy in movimentos:
                novo_x = self.x + dx
                novo_y = self.y + dy
                if not self.ambiente.sala_obstaculo(novo_x, novo_y):
                    self.x = novo_x
                    self.y = novo_y
                    print(f"Agente (modelo) moveu-se para ({self.x}, {self.y}).")
                    break
            self.desempenho -= 1  # Medida 2: -1 por movimento


Visualização do Ambiente

In [4]:
def imprimir_ambiente(ambiente, agente):
    print("\nEstado atual do ambiente:")
    for y in range(ambiente.altura):
        linha = ''
        for x in range(ambiente.largura):
            if (x, y) == (agente.x, agente.y):
                linha += ' A '  # Posição do agente
            else:
                sala = ambiente.salas[(x, y)]
                if sala.obstaculo:
                    linha += ' X '
                elif sala.suja:
                    linha += ' S '
                else:
                    linha += ' L '
        print(linha)


Função do Simulador

In [5]:
def executar_simulacao(AgenteClasse):
    ambiente = Ambiente(LARGURA, ALTURA, SUJEIRA_INICIAL, OBSTACULOS)
    agente = AgenteClasse(ambiente, POSICAO_INICIAL_AGENTE)
    print(f"Iniciando simulação com {AgenteClasse.__name__}...")

    for iteracao in range(NUM_ITERACOES):
        print(f"\nIteração {iteracao + 1}")
        agente.agir()
        # Medida 1: +1 por sala limpa a cada período
        if MEDIDA_DESEMPENHO == 'medida1':
            salas_limpas = sum(1 for sala in ambiente.salas.values() if not sala.suja and not sala.obstaculo)
            agente.desempenho = salas_limpas
            print(f"Desempenho atual: {agente.desempenho}")
        elif MEDIDA_DESEMPENHO == 'medida2':
            print(f"Desempenho atual: {agente.desempenho}")

        if ambiente.todas_salas_limpas():
            print("Todas as salas estão limpas!")
            break
        imprimir_ambiente(ambiente, agente)    
    print(f"Desempenho final do agente: {agente.desempenho}")
    return agente.desempenho


Executando a Simulação com o Agente Reativo Simples

In [6]:
print("Simulação com Agente Reativo Simples")
desempenho_reativo = executar_simulacao(AgenteReativoSimples)


Simulação com Agente Reativo Simples
Iniciando simulação com AgenteReativoSimples...

Iteração 1
Agente moveu-se para (0, 1).
Desempenho atual: -1

Estado atual do ambiente:
 L  L  L  L  L 
 A  L  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 2
Agente moveu-se para (1, 1).
Desempenho atual: -2

Estado atual do ambiente:
 L  L  L  L  L 
 L  A  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 3
Agente moveu-se para (1, 0).
Desempenho atual: -3

Estado atual do ambiente:
 L  A  L  L  L 
 L  L  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 4
Agente moveu-se para (1, 1).
Desempenho atual: -4

Estado atual do ambiente:
 L  L  L  L  L 
 L  A  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 5
Agente moveu-se para (2, 1).
Desempenho atual: -5

Estado atual do ambiente:
 L  L  L  L  L 
 L  L  A  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 6
Agente moveu-se para (1, 1).
Desempenho atual: -6

Estado 

Executando a Simulação com o Agente Baseado em Modelos

In [7]:
print("\n-----------------------------------\n")
print("Simulação com Agente Baseado em Modelos")
desempenho_modelo = executar_simulacao(AgenteBaseadoEmModelo)



-----------------------------------

Simulação com Agente Baseado em Modelos
Iniciando simulação com AgenteBaseadoEmModelo...

Iteração 1
Agente (modelo) moveu-se para (1, 0).
Desempenho atual: -1

Estado atual do ambiente:
 L  A  L  L  L 
 L  L  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 2
Agente (modelo) moveu-se para (0, 1).
Desempenho atual: -2

Estado atual do ambiente:
 L  L  L  L  L 
 A  L  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 3
Agente (modelo) moveu-se para (2, 0).
Desempenho atual: -3

Estado atual do ambiente:
 L  L  A  L  L 
 L  L  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 4
Agente (modelo) moveu-se para (1, 1).
Desempenho atual: -4

Estado atual do ambiente:
 L  L  L  L  L 
 L  A  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 
 L  L  L  L  S 

Iteração 5
Agente (modelo) moveu-se para (1, 1).
Desempenho atual: -5

Estado atual do ambiente:
 L  L  L  L  L 
 L  A  L  L  L 
 L  S  X  L  L 
 L  L  S  X  L 

## Agente Reativo Simples

- **Desempenho Final**: -25
- **Iterações**: 31
- **Salas Limpas**: Todas as salas sujas foram limpas ao final da simulação.
- **Comportamento**:
  - O agente se movimenta aleatoriamente quando a sala atual está limpa.
  - Ao encontrar uma sala suja, ele a limpa imediatamente.
  - Em algumas iterações o agente ficou preso e repetiu movimentos.

## Agente Baseado em Modelos

- **Desempenho Final**: -46
- **Iterações**: 50 (limite máximo definido)
- **Salas Limpas**: Nem todas as salas sujas foram limpas.
- **Comportamento**:
  - O agente mantém um modelo interno do ambiente.
  - Visita salas adjacentes não exploradas e as adiciona à lista de salas para visitar.
  - Em muitas iterações, após esgotar as salas conhecidas principalmente, o agente repetiu movimentos entre as mesmas salas.
  - As vezes o agente parece ficar preso em um loop, se movimentando entre salas que já foram visitadas.


## Comparação entre os Agentes

- **Eficiência**:
  - O agente reativo simples, mesmo sendo básico, acabou sendo mais eficiente nessa simulação específica. Ele conseguiu limpar as salas de maneira mais direta.
  - O agente baseado em modelos, apesar de parecer mais avançado, não conseguiu explorar de forma eficiente e teve um desempenho pior.

- **Complexidade**:
  - O agente baseado em modelos é claramente mais complicado em termos de lógica, mas essa complexidade não resultou em um desempenho melhor.

- **Adequação ao Ambiente**:
  - Em ambientes onde a visibilidade é limitada ou onde o agente não conhece todo o mapa, é fundamental ter uma estratégia de exploração bem planejada.
  - Por outro lado, o agente reativo pode ser suficiente em cenários mais simples.

## Possíveis melhorias

- **Usar algoritmos como BFS ou DFS para explorar o ambiente de forma completa.**:
  - Talvez também implementar um algoritmo de caminho (como A*) para encontrar a rota mais curta até a sala suja conhecida.

- **Mapeamento Completo do Ambiente**:
  - Armazenar informações sobre salas adjacentes, mesmo se não forem imediatamente acessíveis e marcar salas como visitadas, sujas, limpas ou obstáculos.
  - Mantendo esse registro podemos evitar loops (se for detectado que está em um loop, explorar uma direção diferente / mudar estratégia) e priorizar salas sujas


## Conclusões

- **Importância da Estratégia de Exploração**:
  - Quando os agentes estão em ambientes que não conseguem observar completamente ou desconhecem, o jeito como eles exploram o ambiente faz toda a diferença no desempenho.
  - Um modelo interno, por si só, não adianta muito se não tiver uma estratégia eficiente de busca junto com ele. (pra melhorar poderia implementar uma DFS ou BFS por exemplo)

- **Necessidade de Melhorias no Agente Baseado em Modelos**:
  - É necessário melhorar os algoritmos de busca e de exploração, tornando-os mais robustos e eficientes.
  - Além disso, o modelo interno precisa realmente guiar o agente de maneira mais inteligente, ajudando nas decisões.

- **Análise do Desempenho**:
  - A métrica de desempenho utilizada (medida 2) penaliza movimentos desnecessários, o que prejudicou bastante o agente baseado em modelos.
  - Já o agente reativo simples, mesmo sendo menos complexo, conseguiu um desempenho melhor ao encontrar todas as salas sujas com menos movimentos.
